In [1]:
# CLIMADA 6.0.1 Installation and Environment Setup for Google Colab

# =============================================================================
# STEP 1: Install CLIMADA using Official Requirements
# =============================================================================

# Clone CLIMADA repository to access requirements files
!git clone https://github.com/CLIMADA-project/climada_python.git
!cd climada_python && git checkout v6.0.1

# Install CLIMADA dependencies from official requirements
print("Installing CLIMADA dependencies from official requirements...")

# Pandas is giving an issue, so installing the save version of the pandas used in climada
!pip install pandas==2.1.4 --force-reinstall --no-cache-dir

# Note: Restart the session after running this cell


Cloning into 'climada_python'...
remote: Enumerating objects: 37598, done.
remote: Counting objects: 100% (1927/1927), done.
remote: Compressing objects: 100% (621/621), done.
remote: Total 37598 (delta 1627), reused 1309 (delta 1306), pack-reused 35671 (from 4)
Receiving objects: 100% (37598/37598), 201.24 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (28182/28182), done.
Note: switching to 'v6.0.1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 304afb4d 'Automated update v6.0.1'
Installing CLIMADA depende

In [1]:
#Installing condacolab to setup and manage conda environment in colab
!pip install -q condacolab
import condacolab
condacolab.install()


⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [1]:
# Install CLIMADA requirements
print("Installing dependencies from the source package...")
!mamba env update -n base -f climada_python/requirements/env_climada.yml

# Installing fiona package as it is missing in the official environment.yml
print("Installing Fiona package...")
!mamba install fiona==1.10.1

#Installing climada package
!mamba install -y -c conda-forge climada==6.0.1

# Checking if proj database file "proj.db" is available in /usr/local/share/proj. Normally, it should be available.
!ls /usr/local/share/proj/proj.db

'''
Setting up the environment path so that the packages like pyproj, rasterio, and fiona
could locate the proj.db without any error/conflict (CRSError: Missing projection database)
'''

import os
os.environ['PROJ_LIB'] = '/usr/local/share/proj'

print("✓ CLIMADA 6.0.1 installed using official requirements")

# =============================================================================
# STEP 2: Verification and Testing
# =============================================================================

# Test CLIMADA installation
import sys
print("Python version:", sys.version)
print("\n" + "="*50)
print("TESTING CLIMADA INSTALLATION")
print("="*50)

try:
    import climada
    print("✓ CLIMADA imported successfully")
    #print(f"✓ CLIMADA version: {climada.__version__}")
except ImportError as e:
    print("✗ CLIMADA import failed:", e)

# Test key modules
try:
    from climada.entity import Exposures, ImpactFuncSet, MeasureSet
    print("✓ CLIMADA entity modules imported")
except ImportError as e:
    print("✗ CLIMADA entity import failed:", e)

try:
    from climada.hazard import Hazard
    print("✓ CLIMADA hazard module imported")
except ImportError as e:
    print("✗ CLIMADA hazard import failed:", e)

try:
    from climada.engine import Impact
    print("✓ CLIMADA engine module imported")
except ImportError as e:
    print("✗ CLIMADA engine import failed:", e)

# Test geospatial dependencies
try:
    import geopandas as gpd
    import rasterio
    import fiona
    print("✓ Geospatial libraries imported successfully")
except ImportError as e:
    print("✗ Geospatial libraries import failed:", e)

# =============================================================================
# STEP 3: CLIMADA Configuration
# =============================================================================

# Set up CLIMADA configuration
import os
from pathlib import Path

# Create CLIMADA directories
climada_dir = Path("/content/climada_data")
climada_dir.mkdir(exist_ok=True)
(climada_dir / "hazard").mkdir(exist_ok=True)
(climada_dir / "entity").mkdir(exist_ok=True)
(climada_dir / "results").mkdir(exist_ok=True)

# Set environment variables
os.environ['CLIMADA_DATA'] = str(climada_dir)
print(f"✓ CLIMADA data directory set to: {climada_dir}")

# =============================================================================
# STEP 5: Sample CLIMADA Usage
# =============================================================================

# Simple test to verify everything works
print("\n" + "="*50)
print("RUNNING SAMPLE CLIMADA TEST")
print("="*50)

try:
    # Import required modules
    from climada.entity import Exposures
    from climada.hazard import Hazard
    from climada.hazard.centroids import Centroids
    import numpy as np

    # Create a simple exposure
    # exp = Exposures()
    gdf = gpd.GeoDataFrame({
        'value': [1000, 2000, 3000],
        'geometry': gpd.points_from_xy([1, 2, 3], [1, 2, 3])
    })
    #exp.gdf.crs = 'EPSG:4326'

    exp = Exposures(gdf, crs=4326)

    print("✓ Sample exposure created successfully")
    print(f"✓ Exposure shape: {exp.gdf.shape}")

    centroids = Centroids(lat=np.array([1, 2, 3]), lon=np.array([1, 2, 3]))

    # Create a simple hazard
    haz = Hazard()
    haz.haz_type = 'TC'  # Tropical Cyclone
    haz.description = 'Test hazard'
    haz.centroids = centroids
    #haz.centroids.set_lat_lon(np.array([1, 2, 3]), np.array([1, 2, 3]))
    haz.event_id = np.array([1])
    haz.frequency = np.array([1])
    haz.intensity = np.array([[10, 20, 30]])
    haz.fraction = np.array([[1, 1, 1]])

    print("✓ Sample hazard created successfully")
    print(f"✓ Hazard intensity shape: {haz.intensity.shape}")

    print("\n🎉 CLIMADA installation and setup completed successfully!")

except Exception as e:
    print(f"✗ Sample test failed: {e}")

# =============================================================================
# STEP 6: Useful Helper Functions
# =============================================================================

def setup_climada_session():
    """Run this function at the start of each new Colab session"""
    import os
    from pathlib import Path

    # Recreate directories if needed
    climada_dir = Path("/content/climada_data")
    climada_dir.mkdir(exist_ok=True)
    (climada_dir / "hazard").mkdir(exist_ok=True)
    (climada_dir / "entity").mkdir(exist_ok=True)
    (climada_dir / "results").mkdir(exist_ok=True)

    # Set environment variables
    os.environ['CLIMADA_DATA'] = str(climada_dir)

    print("✓ CLIMADA session setup complete")
    print(f"✓ Data directory: {climada_dir}")

def check_climada_status():
    """Check if CLIMADA is properly installed and configured"""
    try:
        import climada
        from climada.entity import Exposures
        from climada.hazard import Hazard
        from climada.engine import Impact

        print("✓ All CLIMADA modules available")
        print(f"✓ CLIMADA version: {climada.__version__}")
        return True
    except ImportError as e:
        print(f"✗ CLIMADA check failed: {e}")
        return False

# Save helper functions for reuse
print("\n" + "="*50)
print("HELPER FUNCTIONS DEFINED")
print("="*50)
print("Use setup_climada_session() to reinitialize CLIMADA in new sessions")
print("Use check_climada_status() to verify installation")

# =============================================================================
# STEP 7: Save Environment Setup Script
# =============================================================================

# Create a script to quickly reinstall CLIMADA in new sessions

setup_script = '''
# Note: It need to restart and re-run the cell two times to make it fully work

# Quick CLIMADA 6.0.1 Setup Script for Google Colab
# Run this cell in new Colab sessions

# Clone CLIMADA repository to access requirements files
!git clone https://github.com/CLIMADA-project/climada_python.git
!cd climada_python && git checkout v6.0.1

# Install CLIMADA dependencies from official requirements
print("Installing CLIMADA dependencies from official requirements...")

# Pandas is giving issue so installing the package to mactch the requirement of climada package
!pip install pandas==2.1.4 --force-reinstall --no-cache-dir



# *** Restart the session is required after running this cell ***

#Installing condacolab to setup and manage conda environment in colab
!pip install -q condacolab
import condacolab
condacolab.install()

# *** Restart the session is required after running this cell ***

# Install CLIMADA requirements
print("Installing dependencies from the source package...")
!mamba env update -n base -f climada_python/requirements/env_climada.yml

# Installing fiona package as it is missing in the official environment.yml
print("Installing Fiona package...")
!mamba install fiona==1.10.1


# Checking if proj database file "proj.db" is available in /usr/local/share/proj. Normally, it should be available.
!ls /usr/local/share/proj/proj.db

#Setting up the environment path so that the packages like pyproj, rasterio, and fiona
#could locate the proj.db without any error/conflict (CRSError: Missing projection database)

import os
os.environ['PROJ_LIB'] = '/usr/local/share/proj'

import climada
from climada.entity import Exposures
from climada.hazard import Hazard
from climada.engine import Impact
print("✓ CLIMADA Package is installed successfully")

'''

with open('/content/quick_climada_setup.py', 'w') as f:
    f.write(setup_script)

print("\n✓ Quick setup script saved to /content/quick_climada_setup.py")
print("✓ Run this script in new Colab sessions for faster setup")

print("\n" + "="*60)
print("🎉 CLIMADA 6.0.1 ENVIRONMENT SETUP COMPLETE!")
print("="*60)


Installing dependencies from the source package...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



pillow-11.3.0        | 41.1 MB   | :   0% 0/1 [00:00<?, ?it/s]
llvmlite-0.44.0      | 28.6 MB   | :   0% 0/1 [00:00<?, ?it/s]

scipy-1.14.1         | 16.9 MB   | :   0% 0/1 [00:00<?, ?it/s]


pandas-2.1.4         | 14.3 MB   | :   0% 0/1 [00:00<?, ?it/s]



icu-75.1             | 11.6 MB   | :   0% 0/1 [00:00<?, ?it/s]




statsmodels-0.14.5   | 11.5 MB   | :   0% 0/1 [00:00<?, ?it/s]





libgdal-core-3.10.3  | 10.3 MB   | :   0% 0/1 [00:00<?, ?it/s]






scikit-learn-1.7.0   | 10.0 MB   | :   0% 0/1 [00:00<?, ?it/s]







libarrow-20.0.0      | 8.8 MB    | :   0% 0/1 [00:00<?, ?it/s]








matplotlib-base-3.10 | 7.9 M